In [1]:
%matplotlib inline
import json
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize
import time
import numpy as np
import seaborn as sns
import re

In [2]:
path = "executions-bkp-gcloudeu-28-11.jsonl"
raw_data = pd.read_json(path, lines=True)
raw_data.head()

,_id,topic,payload,timestamp
0,{'$oid': '5d274d843299fa6236b8bc28'},data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...,{'responseStamp': 1},2019-07-11 14:53:56.866
1,{'$oid': '5d274d853299fa6236b8bc29'},data/case/332f02b9-093e-4a2f-8d93-28022c275e8b...,"{'name': 'Zombie Venom 09', 'source': '# 1 (ca...",2019-07-11 14:53:57.587
2,{'$oid': '5d274e633299fa6236b8bc2a'},data/case/00f14675-854c-4c58-95eb-8dd414f525d3...,{'responseStamp': 1},2019-07-11 14:57:39.979
3,{'$oid': '5d274e643299fa6236b8bc2b'},data/case/00f14675-854c-4c58-95eb-8dd414f525d3...,"{'name': 'Zombie Venom 08', 'source': '# 1 (ca...",2019-07-11 14:57:40.069
4,{'$oid': '5d274e643299fa6236b8bc2c'},data/theme/zombie.note/get,{'responseStamp': 2},2019-07-11 14:57:40.085


In [3]:
nl_experiment = raw_data.loc[lambda df: df.timestamp > '2019-11-28 7:30:00']
nl_experiment.to_json('nl_experiment1.json', orient='records')

In [34]:
def reconstruct_users_track(log):
    users_tracks = dict()
    for index, row in log.iterrows():
        payload = row.payload
        try:
            user = payload['track']['userid'] 
            if user in users_tracks:
                users_tracks[user].append(row)
            else:
                users_tracks.update({user: [row]})
        except:
            pass
    return users_tracks

In [5]:
nl_experiment.head(10)

,_id,topic,payload,timestamp
22113,{'$oid': '5ddf8655cd683579f92534ee'},data/user/login,"{'email': 'A33@ds4h.org', 'password': 'a33', '...",2019-11-28 08:33:25.387
22114,{'$oid': '5ddf8655cd683579f92534ef'},data/user/login/response/2,"{'userid': 93, 'token': 'eyJhbGciOiJIUzI1NiIsI...",2019-11-28 08:33:25.551
22115,{'$oid': '5ddf8655cd683579f92534f0'},data/case/*/list,"{'filterBy': 'user', 'filter': 93, 'responseSt...",2019-11-28 08:33:25.614
22116,{'$oid': '5ddf8655cd683579f92534f1'},data/case/*/list/response/3,[{'id': 'a46c78ec-9973-4792-a4ec-d9a9ef0e4978'...,2019-11-28 08:33:25.729
22117,{'$oid': '5ddf8655cd683579f92534f2'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,{'responseStamp': 4},2019-11-28 08:33:25.754
22118,{'$oid': '5ddf8655cd683579f92534f3'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,"{'name': 'EKG-First', 'source': 'EKG-Descripti...",2019-11-28 08:33:25.883
22119,{'$oid': '5ddf8656cd683579f92534f4'},knot/<</navigate,{},2019-11-28 08:33:26.004
22120,{'$oid': '5ddf8656cd683579f92534f5'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'parameter': 'EKG first', 'track': {'userid':...",2019-11-28 08:33:26.033
22121,{'$oid': '5ddf8656cd683579f92534f6'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.036
22122,{'$oid': '5ddf8656cd683579f92534f7'},2019-11-28T08:33:25.342Z-980cae4f-15d1-9d08-11...,"{'track': {'userid': 93, 'caseid': 'a46c78ec-9...",2019-11-28 08:33:26.082


In [6]:
nl_experiment

,_id,topic,payload,timestamp
22113,{'$oid': '5ddf8655cd683579f92534ee'},data/user/login,"{'email': 'A33@ds4h.org', 'password': 'a33', '...",2019-11-28 08:33:25.387
22114,{'$oid': '5ddf8655cd683579f92534ef'},data/user/login/response/2,"{'userid': 93, 'token': 'eyJhbGciOiJIUzI1NiIsI...",2019-11-28 08:33:25.551
22115,{'$oid': '5ddf8655cd683579f92534f0'},data/case/*/list,"{'filterBy': 'user', 'filter': 93, 'responseSt...",2019-11-28 08:33:25.614
22116,{'$oid': '5ddf8655cd683579f92534f1'},data/case/*/list/response/3,[{'id': 'a46c78ec-9973-4792-a4ec-d9a9ef0e4978'...,2019-11-28 08:33:25.729
22117,{'$oid': '5ddf8655cd683579f92534f2'},data/case/a46c78ec-9973-4792-a4ec-d9a9ef0e4978...,{'responseStamp': 4},2019-11-28 08:33:25.754
...,...,...,...,...
104931,{'$oid': '5ddf9024cd683579f9267870'},2019-11-28T09:15:15.570Z-c73e9cbd-85e8-e63d-93...,"{'responseStamp': 8, 'track': {'userid': 116, ...",2019-11-28 09:15:16.547
104932,{'$oid': '5ddf9024cd683579f9267871'},2019-11-28T09:15:15.570Z-c73e9cbd-85e8-e63d-93...,"{'content': '<div class=""styt-main-frame""> ...",2019-11-28 09:15:16.584
104933,{'$oid': '5ddf9024cd683579f9267872'},2019-11-28T09:15:15.570Z-c73e9cbd-85e8-e63d-93...,"{'track': {'userid': 116, 'caseid': '9b1651f3-...",2019-11-28 09:15:16.624
104934,{'$oid': '5ddf9025cd683579f9267873'},data/case/*/list,{'responseStamp': 2},2019-11-28 09:15:17.504


In [35]:
users_track = reconstruct_users_track(nl_experiment)

In [53]:
print(sorted(users_track.keys()))

[1, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116]


In [42]:
def split_groups(users_track):
    group_A = dict()
    group_B = dict()
    
    for user in users_track.keys():
        for log in users_track[user]:
            try:
                if log['payload']['parameter'] == "Description first":
                    group_A[user] = users_track[user]
                if log['payload']['parameter'] == "EKG first":
                    group_B[user] = users_track[user]
            except:
                pass
    return group_A, group_B

In [50]:
group_A, group_B = split_groups(users_track)
print("Users Group A - Description First")
print(sorted(list(group_A.keys())))
print("Users Group B - EKG First")
print(sorted(list(group_B.keys())))

Users Group A - Description First
[1, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87]
Users Group B - EKG First
[86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116]


In [51]:
# Manually adding user 89 whose don't have Group parameter
# group_B[89]=users_track[89]
#Deleting admin user and user in both groups
del(group_A[1])
del(group_A[87])
del(group_B[87])

In [52]:
print(len(list(group_A.keys())))
print(len(list(group_B.keys())))

24
28


In [12]:
def users_answers(users_track):
    
    users_answers = dict()
    
    for user in users_track.keys():
        users_answers[user] = pd.DataFrame(columns=["Variable", "Answer", "CaseId"])
        
        for log in users_track[user]:
            if re.search('/var/[\w+.]+/set', log['topic']):
                
                var_topic = re.search('/var/[\w+.]+/set', log['topic']).group()
                answer = log['payload']['content']
                case_id = log['payload']['track']['caseid']
                row = {"Variable": var_topic, "Answer": answer, "CaseId": case_id}
                users_answers[user] = users_answers[user].append(row, ignore_index=True)
                
    return users_answers

In [54]:
answers_groupA = users_answers(group_A)
answers_groupB = users_answers(group_B)

In [15]:
def clean_variables_track(answers_track, columns):
    
    users_answers_clean = dict()
    
    for user in answers_track.keys():
        cleaned_track = pd.DataFrame(columns=["Variable", "Answer", "CaseId"])
        group_track = answers_track[user].groupby(columns)
        
        for name, group in group_track:
            cleaned_track = cleaned_track.append(group_track.get_group(name).iloc[-1])
        
        users_answers_clean[user] = cleaned_track.sort_values(by=["CaseId","Variable"], ascending=False)
    
    return users_answers_clean

In [85]:
groupA_clean = clean_variables_track(answers_groupA, columns=["Variable", "CaseId"])
groupB_clean = clean_variables_track(answers_groupB, columns=["Variable", "CaseId"])

In [61]:
cases_dict = {"Case-01":"5c0cf92e-ebe3-4f83-bb3b-11846d0f3da8",
              "Case-02":"71cb08b0-aa66-402f-8d35-027e01bd541f",
              "Case-03":"1edff734-1090-4d44-bd52-963156b6b41d",
              "Case-07":"9c5b0ab8-33c8-4bcf-bcb4-d888327b870b",
              "Case-08":"28688ea2-f8d8-4664-80f6-0926720990fb",
              "Case-09":"ec4be4fd-4e48-4e84-ba8e-36c53b415698",
              "Case-12":"280fa4a7-1e79-4097-a961-7fda24d7d9bf",
              "Case-13":"7fbf35be-79fa-4087-94cd-ccc8030ac305",
              "Case-15":"a775bf21-b15c-44d1-a120-61611bc7960c",
              "Case-16":"b90056f9-e5e2-4281-bcfe-75ab62033b2b",
              "Case-17":"070e904b-fe21-4629-83b2-4f9ea49bccce",
              "Case-19":"64634e19-ea0a-4060-9656-1252b5d6acb5"
             }
cases_name = pd.DataFrame(cases_dict.items(), columns=["Case", "CaseId"])

In [57]:
topic_dict = {"EKG Hypothesis":"/var/EKG_Cycle.Generate_hypothesis.hypothesis/set",
              "EKG Confidence":"/var/EKG_Cycle.Generate_hypothesis.confidence/set",
              "Description Hypothesis":"/var/Description_Cycle.Generate_hypothesis.hypothesis/set",
              "Description Confidence":"/var/Description_Cycle.Generate_hypothesis.confidence/set"
             }
topic_names = pd.DataFrame(topic_dict.items(), columns=["Question", "Variable"])

In [84]:
def beautify_case_topic_name(users_track, topic_names, cases_name):
    
    for user in users_track.keys():
        df_merged = pd.merge(users_track[user],cases_name, on="CaseId", how="outer")
        df_merged = pd.merge(df_merged,topic_names, on="Variable", how="outer")
        df_merged.drop(columns=["Variable", "CaseId"], inplace=True)
        df_merged = df_merged[["Question","Answer","Case"]]
        df_merged = df_merged.sort_values(by=["Case","Question"], ascending=False)
        users_track[user] = df_merged
    
    return users_track

In [86]:
groupA_clean = beautify_case_topic_name(groupA_clean, topic_names, cases_name)

In [88]:
groupB_clean = beautify_case_topic_name(groupB_clean, topic_names, cases_name)

In [130]:
def all_answers_group(users_track, start_df=None):
    
    all_answers_group = pd.DataFrame(columns=["Question","Answer","Case"])
    
    if start_df is not None:
        all_answers_group = start_df
    
    for user in users_track:
        all_answers_group = pd.concat([all_answers_group, users_track[user]])
    
    return all_answers_group

In [127]:
all_answers_groupA = all_answers_group(groupA_clean)

In [133]:
all_answers_groupA

,Question,Answer,Case
6,EKG Hypothesis,Iap,Case-19
18,EKG Confidence,79,Case-19
30,Description Hypothesis,Iap,Case-19
42,Description Confidence,76,Case-19
11,EKG Hypothesis,Ap,Case-17
...,...,...,...
41,Description Confidence,52,Case-02
7,EKG Hypothesis,Nstemi,Case-01
19,EKG Confidence,60,Case-01
31,Description Hypothesis,Nstemi,Case-01


In [128]:
all_answers_groupB = all_answers_group(groupB_clean)

In [134]:
all_answers_groupB

,Question,Answer,Case
6,EKG Hypothesis,Atypisch,Case-19
18,EKG Confidence,68,Case-19
11,EKG Hypothesis,Ischemie,Case-17
23,EKG Confidence,68,Case-17
34,Description Hypothesis,Ap,Case-17
...,...,...,...
18,Description Confidence,81,Case-02
3,EKG Hypothesis,Diffuus vlakke ST segmenten,Case-01
9,EKG Confidence,38,Case-01
14,Description Hypothesis,Instabiele AP,Case-01


In [131]:
all_answers_experiment = all_answers_group(groupB_clean,start_df=all_answers_groupA)

In [132]:
all_answers_experiment

,Question,Answer,Case
6,EKG Hypothesis,Iap,Case-19
18,EKG Confidence,79,Case-19
30,Description Hypothesis,Iap,Case-19
42,Description Confidence,76,Case-19
11,EKG Hypothesis,Ap,Case-17
...,...,...,...
18,Description Confidence,81,Case-02
3,EKG Hypothesis,Diffuus vlakke ST segmenten,Case-01
9,EKG Confidence,38,Case-01
14,Description Hypothesis,Instabiele AP,Case-01


In [135]:
hypothesis_groupA = all_answers_groupA[all_answers_groupA["Question"].str.contains("Hypothesis", na=False)]
hypothesis_groupB = all_answers_groupB[all_answers_groupB["Question"].str.contains("Hypothesis", na=False)]
all_hypothesis = all_answers_experiment[all_answers_experiment["Question"].str.contains("Hypothesis", na=False)]
confidence_groupA = all_answers_groupA[all_answers_groupA["Question"].str.contains("Confidence", na=False)]
confidence_groupB = all_answers_groupB[all_answers_groupB["Question"].str.contains("Confidence", na=False)]
all_confidence = all_answers_experiment[all_answers_experiment["Question"].str.contains("Confidence", na=False)]

In [150]:
answers_bycase_groupA = hypothesis_groupA.groupby("Case").apply(lambda x: x["Answer"].unique())
answers_bycase_groupB = hypothesis_groupB.groupby("Case").apply(lambda x: x["Answer"].unique())
answers_bycase = all_hypothesis.groupby("Case").apply(lambda x: x["Answer"].unique())

In [157]:
answers_bycase_groupA

Case
Case-01    [Acs, ACS, Nste-acs, NSTE-ACS, Acs-nsteni, Acs...
Case-02    [Gastro esofagaal, Gastro-esofagaal, Maagklach...
Case-03    [Tendomyalgeen, Hyperventilatie, Pericarditis ...
Case-07    [Pericarditis, Pericarditis , Pericaridits, Co...
Case-08    [Ap, ACS, NSTE-ACS, IAP, Acs-nstemi, STEMI pro...
Case-09    [Pneumonie, ACS, STEMI, NSTE-ACS, STEMI latera...
Case-12    [Hvs, Costomyalgeen, ACS, Reflux, Reflux/maagk...
Case-13    [Tendomyalgeen, ACS, Costomyalgeen, AP, Ap-kla...
Case-15    [IAp, Iap, ACS, NSTE-ACS, Infarct, Acs-nstemi,...
Case-16    [Ap, ACS, AP, NSTE-ACS panischemie, Iap, Acs-n...
Case-17    [Ap, Oesofagus spasme, ACS, Hyperventilatie, T...
Case-19    [Iap, ACS, Nste-acs, Acs, NSTEMI, Gastritis, G...
dtype: object

In [158]:
answers_bycase_groupA.to_excel("hypothesis_groupA_bycase.xlsx")

In [153]:
answers_bycase_groupB

Case
Case-01    [Nstemi, Doorgemaakt inferolateraal infarct, A...
Case-02    [Ge reflux, Inferior STEMI, Reflux, Pericardit...
Case-03    [Lvh, Hyperventilatie , STEMI, Costo-myalgeen,...
Case-07    [Pericarditis, STEMI, Pericarditis , Acs, Stem...
Case-08    [Infarct, Iap, STEMI septaal, NSTEMI, Panische...
Case-09    [Longembolie, Misschien STEMI lateraal, Doorge...
Case-12    [Atypisch, Geen ischemie, NSTEMI, Niet cardiaa...
Case-13    [La dilatatie, Nstemi, NSTEMI, Mitralisklepins...
Case-15    [Normaal, Iap, Normaal ECG, mogelijk inferior ...
Case-16    [Nstemi, Acs, Hoofdstamstenose, demand ischemi...
Case-17    [Ischemie, Ap, NSTEMI, Doorgemaakt ipl infarct...
Case-19    [Atypisch, Doorgemaakte ischemie inferior, NST...
dtype: object

In [159]:
answers_bycase_groupB.to_excel("hypothesis_groupB_bycase.xlsx")

In [155]:
answers_bycase

Case
Case-01    [Acs, ACS, Nste-acs, NSTE-ACS, Acs-nsteni, Acs...
Case-02    [Gastro esofagaal, Gastro-esofagaal, Maagklach...
Case-03    [Tendomyalgeen, Hyperventilatie, Pericarditis ...
Case-07    [Pericarditis, Pericarditis , Pericaridits, Co...
Case-08    [Ap, ACS, NSTE-ACS, IAP, Acs-nstemi, STEMI pro...
Case-09    [Pneumonie, ACS, STEMI, NSTE-ACS, STEMI latera...
Case-12    [Hvs, Costomyalgeen, ACS, Reflux, Reflux/maagk...
Case-13    [Tendomyalgeen, ACS, Costomyalgeen, AP, Ap-kla...
Case-15    [IAp, Iap, ACS, NSTE-ACS, Infarct, Acs-nstemi,...
Case-16    [Ap, ACS, AP, NSTE-ACS panischemie, Iap, Acs-n...
Case-17    [Ap, Oesofagus spasme, ACS, Hyperventilatie, T...
Case-19    [Iap, ACS, Nste-acs, Acs, NSTEMI, Gastritis, G...
dtype: object

In [156]:
answers_bycase.to_excel("hypothesis_experiment_bycase.xlsx")

In [167]:
all_confidence.astype({'Answer':'float64'})

,Question,Answer,Case
18,EKG Confidence,79.0,Case-19
42,Description Confidence,76.0,Case-19
23,EKG Confidence,18.0,Case-17
47,Description Confidence,62.0,Case-17
13,EKG Confidence,68.0,Case-16
...,...,...,...
21,Description Confidence,73.0,Case-03
8,EKG Confidence,55.0,Case-02
18,Description Confidence,81.0,Case-02
9,EKG Confidence,38.0,Case-01


In [194]:
mean_std_all = all_confidence.groupby(["Case","Question"]).apply(lambda df: (df.Answer.astype('float64').mean(), df.Answer.astype('float64').std()))

In [195]:
mean_std_all

Case     Question              
Case-01  Description Confidence                 (58.25, 19.229139448284435)
         EKG Confidence             (59.73076923076923, 18.151989449239693)
Case-02  Description Confidence     (69.66666666666667, 15.367064347710224)
         EKG Confidence              (63.17307692307692, 17.54961722821076)
Case-03  Description Confidence      (67.6470588235294, 19.399818070705475)
         EKG Confidence             (60.68627450980392, 23.559703050826787)
Case-07  Description Confidence     (72.26923076923077, 18.985149391982606)
         EKG Confidence              (69.42307692307692, 21.86890186493781)
Case-08  Description Confidence                 (72.75, 14.215256923874598)
         EKG Confidence              (68.73076923076923, 22.35113313319614)
Case-09  Description Confidence                 (58.26, 22.642573229405254)
         EKG Confidence              (55.80392156862745, 26.45147981330582)
Case-12  Description Confidence     (56.91836734693877, 

In [196]:
mean_std_all.to_excel("Mean-Std-all.xlsx")

In [197]:
mean_std_groupA = confidence_groupA.groupby(["Case","Question"]).apply(lambda df: (df.Answer.astype('float64').mean(), df.Answer.astype('float64').std()))

In [198]:
mean_std_groupA

Case     Question              
Case-01  Description Confidence      (54.541666666666664, 20.3811062691313)
         EKG Confidence                          (61.5, 17.433101946534382)
Case-02  Description Confidence                  (68.25, 16.04138669057391)
         EKG Confidence             (66.41666666666667, 15.021482201708054)
Case-03  Description Confidence     (70.78260869565217, 16.681390071080926)
         EKG Confidence              (73.26086956521739, 17.76671400350142)
Case-07  Description Confidence     (65.70833333333333, 19.116301185493302)
         EKG Confidence             (82.04166666666667, 18.355141837753745)
Case-08  Description Confidence      (74.29166666666667, 12.21360313131532)
         EKG Confidence              (83.95833333333333, 14.97382982774096)
Case-09  Description Confidence    (58.458333333333336, 21.500210649456776)
         EKG Confidence              (65.08333333333333, 27.29057277374076)
Case-12  Description Confidence     (55.30434782608695, 

In [199]:
mean_std_groupA.to_excel("Mean-Std-GroupA.xlsx")

In [200]:
mean_std_groupB = confidence_groupB.groupby(["Case","Question"]).apply(lambda df: (df.Answer.astype('float64').mean(), df.Answer.astype('float64').std()))

In [201]:
mean_std_groupB

Case     Question              
Case-01  Description Confidence     (61.42857142857143, 17.941114733583245)
         EKG Confidence            (58.214285714285715, 18.929414975713097)
Case-02  Description Confidence     (70.92592592592592, 14.932993738507427)
         EKG Confidence            (60.392857142857146, 19.291793084710413)
Case-03  Description Confidence      (65.07142857142857, 21.32973680000932)
         EKG Confidence             (50.357142857142854, 22.92049751530726)
Case-07  Description Confidence     (77.89285714285714, 17.276272072399667)
         EKG Confidence             (58.607142857142854, 18.80177155899448)
Case-08  Description Confidence     (71.42857142857143, 15.833458646120647)
         EKG Confidence             (55.67857142857143, 19.231635918606095)
Case-09  Description Confidence     (58.07692307692308, 24.073093821813725)
         EKG Confidence             (47.55555555555556, 23.163853361738038)
Case-12  Description Confidence      (58.34615384615385,

In [202]:
mean_std_groupB.to_excel("Mean-Std-GroupB.xlsx")

In [91]:
def save_as_xlsx(users_dict, path, filename_p=""):
    
    filename = path + filename_p
    
    for user in users_dict.keys():
        
        filename += str(user) + ".xlsx"
        users_dict[user].to_excel(filename, index=False)
        filename = path + filename_p

In [92]:
path = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/GroupA/"
save_as_xlsx(groupA_clean, path)

In [93]:
path_B = "/home/marcosf/git/harena-analytics/resources/nl-experiment1/Tables/GroupB/"
save_as_xlsx(groupB_clean, path_B)